# Manual Convolution Exercise

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/USER/teaching-cnn/blob/main/book/exercises/manual_convolution.ipynb)

Implement 2D convolution from scratch and compare to SciPy.

In [ ]:
# Setup
import numpy as np
from numpy.typing import NDArray
try:
    import scipy.signal as sps
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'scipy', '-q'])
    import scipy.signal as sps

In [ ]:
def conv2d(image: NDArray[np.float32], kernel: NDArray[np.float32], padding: str = 'valid', stride: int = 1) -> np.ndarray:
    """
    Perform a 2D convolution using nested loops.

    Parameters
    ----------
    image : ndarray
        Input 2D image.
    kernel : ndarray
        2D convolution kernel (will be flipped).
    padding : {'valid', 'same'}
        Padding mode.
    stride : int
        Stride for both dimensions.

    Returns
    -------
    ndarray
        Convolved output.
    """
    if kernel.ndim != 2 or image.ndim != 2:
        raise ValueError('Both image and kernel must be 2D.')
    k = np.flipud(np.fliplr(kernel))
    if padding not in {'valid','same'}:
        raise ValueError("padding must be 'valid' or 'same'")
    pad_h = pad_w = 0
    if padding == 'same':
        pad_h = (k.shape[0] - 1) // 2
        pad_w = (k.shape[1] - 1) // 2
    img = np.pad(image, ((pad_h,pad_h),(pad_w,pad_w)), mode='constant')
    out_h = (img.shape[0] - k.shape[0]) // stride + 1
    out_w = (img.shape[1] - k.shape[1]) // stride + 1
    out = np.zeros((out_h, out_w), dtype=np.float32)
    for i in range(0, out_h):
        for j in range(0, out_w):
            region = img[i*stride:i*stride+k.shape[0], j*stride:j*stride+k.shape[1]]
            out[i,j] = float(np.sum(region * k))
    return out

In [ ]:
# Quick check against SciPy
np.random.seed(0)
img = np.random.rand(8,8).astype(np.float32)
ker = np.array([[1,2,1],[0,0,0],[-1,-2,-1]], dtype=np.float32)
ref = sps.convolve2d(img, ker, mode='same')
out = conv2d(img, ker, padding='same')
print('Max abs diff:', np.max(np.abs(ref - out)))
assert np.allclose(ref, out, atol=1e-5)
print('OK')